In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
from numpy import log, exp
from scipy.stats import lognorm
from scipy.optimize import minimize
from matplotlib import use, get_backend
from harmonize_preds import aux_functions


#use('TkAgg', force=True)
from matplotlib import pyplot as plt
#print("Switched to:", get_backend())

Load preds: 

In [2]:
df_preds= pd.read_csv('predictions/preds_2nd_sprint.csv.gz')

df_preds = df_preds.loc[df_preds.model_id != 141]
df_preds.date = pd.to_datetime(df_preds.date)

df_preds = df_preds.loc[df_preds.date <= Week(2025, 40).startdate().strftime('%Y-%m-%d')]
            
df_preds.head()

/var/folders/ch/kxpr39wx44v97968yr_4hmch0000gn/T/ipykernel_44647/940469928.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_preds= pd.read_csv('preds_2nd_sprint.csv.gz')


,date,lower_95,lower_90,lower_80,lower_50,pred,upper_50,upper_80,upper_90,upper_95,valid_test,state,model_id
0,2024-10-06,22.422222,40.304679,79.162970,244.274988,853.700012,2982.915845,9196.810733,18041.732884,32366.797486,3,RS,108
1,2024-10-13,18.743651,34.187301,68.269130,216.515891,779.997873,2809.263104,8900.886361,17748.532118,32294.740637,3,RS,108
2,2024-10-20,6.581569,12.226861,24.874887,81.165077,301.351891,1118.132411,3638.484729,7371.869282,13600.483733,3,RS,108
3,2024-10-27,17.097895,32.060474,66.082749,220.934380,843.905436,3222.679992,10763.205908,22149.610067,41420.387215,3,RS,108
4,2024-11-03,18.724941,35.572182,74.438413,255.270334,1003.039414,3940.407380,13500.169745,28209.239339,53454.977878,3,RS,108


### Estimate the log normal parameters from the CDF: 

Fixed conditions for the entire method:

In [3]:
l_pos = tuple([col for col in df_preds.columns if 'lower' in col])
u_pos = tuple([col for col in df_preds.columns if 'upper' in col])
the_pos = list(l_pos + u_pos)
l_pos = list(l_pos)
u_pos = list(u_pos)

lvls = (0.5, 0.8, 0.9, 0.95)
p = [aux_functions.quantile_pair(lvl) for lvl in lvls]
p = [i for quantile in p for i in quantile]
p.append(0.5)
ps = p.copy()
ps.sort()


In [4]:
ps

[0.025000000000000022,
 0.04999999999999999,
 0.09999999999999998,
 0.25,
 0.5,
 0.75,
 0.9,
 0.95,
 0.975]

In [5]:
the_pos

['lower_95',
 'lower_90',
 'lower_80',
 'lower_50',
 'upper_50',
 'upper_80',
 'upper_90',
 'upper_95']

In [1]:
def process_row(row):
    '''
    Estimate the lognormal parameters for each row in the dataframe
    '''
    #the_pos =  [ 'upper_50', 'upper_80', 'upper_90', 'upper_95']

    #ps = [0.5, 0.75,0.9,0.95, 0.975]
    # pegar valores da linha
    xi_values = list(row[the_pos].values) if hasattr(row[the_pos], "values") else [row[the_pos]]
    xi_values.append(row["pred"])
    
    # ordenar
    xi_sorted = sorted(xi_values)

    out = pd.DataFrame({
    'p': ps,
    'xi': xi_sorted
    })

    out = out.loc[out.xi > 0]
    # ajustar CDF (aqui você precisa garantir que 'out' está definido de fora
    cdf_fit = aux_functions.fit_ln_CDF(
        x=out["xi"],
        Fhat=out["p"],
        weighting=1
    )
    
    return cdf_fit

def process_row2(row):
    #the_pos =  [ 'upper_50', 'upper_80', 'upper_90', 'upper_95']

    #ps = [0.5, 0.75,0.9,0.95, 0.975]
    # pegar valores da linha
    xi_values = list(row[the_pos].values) if hasattr(row[the_pos], "values") else [row[the_pos]]
    xi_values.append(row["pred"])
    
    # ordenar
    xi_sorted = sorted(xi_values)

    out = pd.DataFrame({
    'p': ps,
    'xi': xi_sorted
    })

    # ajustar CDF (aqui você precisa garantir que 'out' está definido de fora
    cdf_fit = fit_ln_CDF(
        x=out["xi"],
        Fhat=out["p"],
        weighting=1
    )
    
    return cdf_fit

In [9]:
%%time 

df_preds[['mu', 'sigma']] = df_preds.apply(process_row, axis=1, result_type = 'expand')

df_preds.head()

CPU times: user 37min 38s, sys: 11.4 s, total: 37min 50s
Wall time: 38min 14s


,date,lower_95,lower_90,lower_80,lower_50,pred,upper_50,upper_80,upper_90,upper_95,valid_test,state,model_id,mu,sigma
0,2024-10-06,22.422222,40.304679,79.162970,244.274988,853.700012,2982.915845,9196.810733,18041.732884,32366.797486,3,RS,108,6.748348,1.855412
1,2024-10-13,18.743651,34.187301,68.269130,216.515891,779.997873,2809.263104,8900.886361,17748.532118,32294.740637,3,RS,108,6.657830,1.900458
2,2024-10-20,6.581569,12.226861,24.874887,81.165077,301.351891,1118.132411,3638.484729,7371.869282,13600.483733,3,RS,108,5.704178,1.945792
3,2024-10-27,17.097895,32.060474,66.082749,220.934380,843.905436,3222.679992,10763.205908,22149.610067,41420.387215,3,RS,108,6.736465,1.987314
4,2024-11-03,18.724941,35.572182,74.438413,255.270334,1003.039414,3940.407380,13500.169745,28209.239339,53454.977878,3,RS,108,6.909363,2.029238


In [10]:
df_preds.to_csv('predictions/preds_CDF.csv', index = False)